In [8]:
import cv2
import numpy as np

point_radius = 10

cap = cv2.VideoCapture(r'.\video_data\point_test.mp4')

initial_frame = cap.read()[1]
prev_frame_gray = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)

point_selected = False
selected_point = ()
prev_points = np.array([[]])

lk_params = dict(
    winSize = (20,20), maxLevel = 2,
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )

def select_point_on_click(event, x, y, _flags, _params):
    global selected_point, point_selected, prev_points

    if event != cv2.EVENT_LBUTTONDOWN:
        return
    
    selected_point = (x, y)
    point_selected = True
    prev_points = np.array([[x, y]], dtype=np.float32)


cv2.namedWindow("Video")
cv2.setMouseCallback("Video", select_point_on_click)

while True:
    curr_frame = cap.read()[1]
    curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    if point_selected:
        cv2.circle(curr_frame, selected_point, point_radius, (0,0,255), 2)
        new_points = cv2.calcOpticalFlowPyrLK(
            prev_frame_gray, curr_frame_gray, prev_points, None, **lk_params
            )[0]
        x, y = new_points.ravel()
        cv2.circle(curr_frame, (int(x),int(y)), point_radius, (0,255,0), -1)

        prev_frame_gray = curr_frame_gray.copy()
        prev_points = new_points

    cv2.imshow('Video', curr_frame)
    key = cv2.waitKey(100) #default framerate 20, usable playback is 100
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()